In [1]:
import pandas as pd
import requests
import json

In [2]:
'''
Пример запроса по одному тикеру.

ticker = 'SNGSP'
columns = 'TRADEDATE,SECID,SHORTNAME,CLOSE'
url = f'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{ticker}\
.json?iss.meta=off&history.columns={columns}'

params = {
    'from': '2023-08-10',
    'till': '2023-08-11',
    'tradingsession': '1'
}

response = requests.get(url, params=params)
print(response.text)

df = json.loads(response.text)
#df['history']
df = pd.DataFrame(data=df.get('history').get('data'), columns=df.get('history').get('columns'))
df

----------------------
Для индексов url меняется:
# IMOEX
url_2 = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/IMOEX\
.json?iss.meta=off&history.columns={columns}'

# РТС
url_3 = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/RTSI/securities/RTSI\
.json?iss.meta=off&history.columns={columns}'

'''

"\nПример запроса по одному тикеру.\n\nticker = 'SNGSP'\ncolumns = 'TRADEDATE,SECID,SHORTNAME,CLOSE'\nurl = f'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{ticker}.json?iss.meta=off&history.columns={columns}'\n\nparams = {\n    'from': '2023-08-10',\n    'till': '2023-08-11',\n    'tradingsession': '1'\n}\n\nresponse = requests.get(url, params=params)\nprint(response.text)\n\ndf = json.loads(response.text)\n#df['history']\ndf = pd.DataFrame(data=df.get('history').get('data'), columns=df.get('history').get('columns'))\ndf\n\n----------------------\nДля индексов url меняется:\n# IMOEX\nurl_2 = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/IMOEX.json?iss.meta=off&history.columns={columns}'\n\n# РТС\nurl_3 = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/RTSI/securities/RTSI.json?iss.meta=off&history.columns={columns}'\n\n"

In [3]:
# функция преобразует файл json, полученный по запросу к api, в датафрейм
def to_df(response_json):
    df = json.loads(response_json.text)
    df = pd.DataFrame(data=df.get('history').get('data'), columns=df.get('history').get('columns'))
    return df

In [4]:
def get_table(my_tickers='SBERP', start_date='2023-08-10', end_date='2023-08-11'):
# даты в формате '2023-08-17'
  
    # параметры для подключения к api
    # чтобы взять только первую и последнюю даты, сделаем 2 параметра
    params_start = {
        'from': start_date,
        'till': start_date,
        'tradingsession': '1'
    }
    params_end = {
        'from': end_date,
        'till': end_date,
        'tradingsession': '1'
    }
    columns = 'TRADEDATE,SECID,SHORTNAME,CLOSE'
    
    data = pd.DataFrame() # пустой датафрейм

    # на выходе таблица с тикерами
    for t in my_tickers:
        ticker = t
        #print(t)
        url = f'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{ticker}\
.json?iss.meta=off&history.columns={columns}'
        
        for p in params_start, params_end: # для первой и второй дат
            response = requests.get(url, params=p) # получили json с информацией по тикеру
            df = to_df(response) # преобразовали в датафрейм
            data = pd.concat([data, df], axis=0, ignore_index=True) # присоединяем таблицу по тикеру к остальным

    #print('Тикеры добавлены.')
    #display(data)

    # добавляем индексы
    # MOEX:
    ticker_moex = 'IMOEX'
    url_moex = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/{ticker_moex}\
    .json?iss.meta=off&history.columns={columns}'
    
    for p in params_start, params_end:
        response_moex = requests.get(url_moex, params=p)
        df = to_df(response_moex)
        data = pd.concat([data, df], axis=0, ignore_index=True)

    # RTS:
    url_rts = f'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/RTSI/securities/RTSI\
    .json?iss.meta=off&history.columns={columns}'
    
    for p in params_start, params_end:
        response_rts = requests.get(url_rts, params=p)
        df = to_df(response_rts)
        data = pd.concat([data, df], axis=0, ignore_index=True)

    #print('Индексы добавлены.')

    try:
        data['TRADEDATE'] = pd.to_datetime(data['TRADEDATE'], format='%Y-%m-%d') # приводим к формату дат
        data = data.rename(columns={'TRADEDATE': 'date', 
                                    'SECID': 'ticker', 
                                    'SHORTNAME': 'name', 
                                    'CLOSE': 'close_price'}) # переименование столбцов
        #print('Формат дат исправлен, столбцы переименованы.')
    except:
        print('Ошибка перевода в формат дат / переименования столбцов.')
    
    #display(data.head(5))
    #data.info()
    return data

In [5]:
# нужные тикеры (акций)
my_tickers = {
    'GAZP': 0.055,
    'NVTK': 0.085,
    'LKOH': 0.06,
    'ROSN': 0.05,
    'SIBN': 0.03,
    'GMKN': 0.06,
    'CHMF': 0.02,
    'MAGN': 0.02,
    'RUAL': 0.03,
    'NLMK': 0.02,
    'SBERP': 0.17,
    'YNDX': 0.05,
    'MAIL': 0.02,
    'MGNT': 0.0165,
    'FIVE': 0.017,
    'OZON': 0.0165,
    'MVID': 0.01,
    'BELU': 0.01,
    'PHOR': 0.07,
    'GCHE': 0.03,
    'SGZH': 0.02,
    'SELG': 0.03,
    'PIKK': 0.01,
    'SMLT': 0.005,
    'LSRG': 0.005,
    'MOEX': 0.04,
    'SPBE': 0.01,
    'RTKMP': 0.02,
    'MTSS': 0.02
}

In [6]:
sum(my_tickers.values())

1.0000000000000002

In [7]:
len(my_tickers.keys())

29

In [8]:
# две даты, на которые выдается информация (не брать выходные - там пусто)
start = '2022-08-15'
end = '2023-08-15'

In [9]:
data = get_table(my_tickers.keys(), start, end)
display(data.head(5))
data.info()

,date,ticker,name,close_price
0,2022-08-15,GAZP,ГАЗПРОМ ао,174.80
1,2023-08-15,GAZP,ГАЗПРОМ ао,176.11
2,2022-08-15,NVTK,Новатэк ао,1065.00
3,2023-08-15,NVTK,Новатэк ао,1621.40
4,2022-08-15,LKOH,ЛУКОЙЛ,3959.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         60 non-null     datetime64[ns]
 1   ticker       60 non-null     object        
 2   name         60 non-null     object        
 3   close_price  60 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.0+ KB


In [17]:
profit = data.pivot_table(index='ticker', columns='date', values='close_price').reset_index()
profit.columns = ['ticker', 'start_price', 'end_price']
profit['profit_prc'] = round((profit['end_price'] / profit['start_price'] - 1), 4)
profit.sort_values(by='profit_prc', ascending=False)

,ticker,start_price,end_price,profit_prc
23,SBERP,119.130,261.010,1.1910
0,BELU,2618.000,5731.000,1.1891
16,OZON,1395.500,2738.500,0.9624
9,MAGN,27.015,51.850,0.9193
1,CHMF,708.200,1305.000,0.8427
24,SELG,44.200,76.570,0.7324
14,NLMK,117.960,197.400,0.6734
26,SIBN,389.900,646.000,0.6568
19,ROSN,335.900,550.200,0.6380
2,FIVE,1334.000,2176.000,0.6312


In [11]:
koef = pd.DataFrame.from_dict(my_tickers, orient='index').reset_index()
koef.columns = ['ticker', 'koef']
koef.head()

,ticker,koef
0,GAZP,0.055
1,NVTK,0.085
2,LKOH,0.060
3,ROSN,0.050
4,SIBN,0.030


In [16]:
profit = profit.merge(koef, on='ticker', how='left').fillna(1)
profit.head()

,ticker,start_price,end_price,profit_prc,koef_x,profit_prc_koef,koef_y
0,BELU,2618.0,5731.00,1.1891,0.010,0.011891,0.010
1,CHMF,708.2,1305.00,0.8427,0.020,0.016854,0.020
2,FIVE,1334.0,2176.00,0.6312,0.017,0.010730,0.017
3,GAZP,174.8,176.11,0.0075,0.055,0.000412,0.055
4,GCHE,2747.0,4126.00,0.5020,0.030,0.015060,0.030


In [13]:
profit['profit_prc_koef'] = profit['profit_prc'] * profit['koef']
profit.head()

,ticker,start_price,end_price,profit_prc,koef,profit_prc_koef
0,BELU,2618.0,5731.00,1.1891,0.010,0.011891
1,CHMF,708.2,1305.00,0.8427,0.020,0.016854
2,FIVE,1334.0,2176.00,0.6312,0.017,0.010730
3,GAZP,174.8,176.11,0.0075,0.055,0.000412
4,GCHE,2747.0,4126.00,0.5020,0.030,0.015060


In [14]:
portfel_profit = profit.query('ticker != "IMOEX" and ticker != "RTSI"')['profit_prc_koef'].sum()
moex_profit = profit.query('ticker == "IMOEX"')['profit_prc_koef'].max()

In [15]:
print(f'Изменение портфеля за период с {start} по {end}: {portfel_profit:.2%}')
print(f'Изменение индекса MOEX: {moex_profit:.2%}')
print(f'Отличие: {portfel_profit - moex_profit:.2%}')

Изменение портфеля за период с 2022-08-15 по 2023-08-15: 51.00%
Изменение индекса MOEX: 43.88%
Отличие: 7.12%
